In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyace import *
from pyace.linearacefit import LinearACEFit, LinearACEDataset

In [2]:
import tensorflow as tf   # TensorFlow registers PluggableDevices here.
print(tf.config.list_physical_devices()) # GPU device is visible to TensorFlow.

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
df0 = pd.read_pickle("./Datasets/ethanol.pckl.gzip", compression="gzip")

In [4]:
df0.shape

(1000, 5)

In [5]:
df0

,ase_atoms,energy,forces,energy_corrected,energy_corrected_per_atom
0,"(Atom('C', [-0.44129481, 0.21679602, 0.3068532...",-4209.543834,"[[1.3248442563472804, -3.6851458462042075, -0....",-44.536766,-4.948530
1,"(Atom('C', [0.41925059, -0.39155484, -0.173166...",-4209.555040,"[[0.3467547761294032, 1.8307119873204736, -1.5...",-44.547972,-4.949775
2,"(Atom('C', [0.36954819, -0.26426992, -0.127073...",-4209.416772,"[[0.9743223219987526, 0.806011425187941, 3.176...",-44.409705,-4.934412
3,"(Atom('C', [-0.18327929, 0.12954131, -0.512697...",-4209.520120,"[[-1.2485091797325352, -0.1873606509338272, 1....",-44.513053,-4.945895
4,"(Atom('C', [0.02811988, 0.40247618, 0.40818284...",-4209.717043,"[[1.151288102970219, -0.002741513772029247, -1...",-44.709975,-4.967775
...,...,...,...,...,...
995,"(Atom('C', [-0.32652388, -0.51658872, -0.03294...",-4209.436457,"[[2.7201568578213875, 3.66578381198858, 2.4756...",-44.429389,-4.936599
996,"(Atom('C', [-0.33627583, -0.50508262, -0.15660...",-4209.498890,"[[-0.9712440517099267, 3.475118895210679, -0.2...",-44.491822,-4.943536
997,"(Atom('C', [0.07746533, -0.35826784, 0.4507782...",-4209.627098,"[[0.12325057119995139, 0.8819713337546561, -0....",-44.620031,-4.957781
998,"(Atom('C', [0.18038169, -0.30347999, -0.353913...",-4209.406960,"[[-0.07392296614804496, 1.7788623335772662, 1....",-44.399892,-4.933321


In [6]:
df0["ase_atoms"]

0      (Atom('C', [-0.44129481, 0.21679602, 0.3068532...
1      (Atom('C', [0.41925059, -0.39155484, -0.173166...
2      (Atom('C', [0.36954819, -0.26426992, -0.127073...
3      (Atom('C', [-0.18327929, 0.12954131, -0.512697...
4      (Atom('C', [0.02811988, 0.40247618, 0.40818284...
                             ...                        
995    (Atom('C', [-0.32652388, -0.51658872, -0.03294...
996    (Atom('C', [-0.33627583, -0.50508262, -0.15660...
997    (Atom('C', [0.07746533, -0.35826784, 0.4507782...
998    (Atom('C', [0.18038169, -0.30347999, -0.353913...
999    (Atom('C', [0.30510098, 0.4347642, -0.42044504...
Name: ase_atoms, Length: 1000, dtype: object

In [7]:
df0["ase_atoms"].map(len).sum()

9000

In [8]:
# split 95% train, 5% test
df_train = df0.sample(frac=0.95, random_state=42)
df_test = df0.loc[(i for i in df0.index if i not in df_train.index)]

In [9]:
len(df_train), len(df_test)

(950, 50)

In [10]:
use_basic_functions = True

# Create empty bbasis configuration
if use_basic_functions:
  bconf = create_multispecies_basis_config(potential_config = {
  "deltaSplineBins": 0.001,
  "elements": ['C', 'O', 'H'],

  "embeddings": {
    "ALL": {
      "npot": 'FinnisSinclairShiftedScaled',
      "fs_parameters": [ 1, 1, 1, 0.5], # embedding function: 1*rho_1^1 + 1*rho_2^0.5
      "ndensity": 2,
    },
  },

  "bonds": {
    "ALL": {
      "radbase": "ChebExpCos",
      "radparameters": [ 5.25 ],
      "rcut": 4,
      "dcut": 0.01,
      "NameOfCutoffFunction": "cos",
      # core-repulsion parameters
      "core-repulsion": [ 0.0, 5.0 ],
    }
  },

  "functions": {
    "ALL": {
        "nradmax_by_orders": [15, 3, 2, 2, 1],
        "lmax_by_orders": [ 0, 2, 2, 1, 1],
    }
  }
})
else:
  bconf = create_multispecies_basis_config(potential_config = {
    "deltaSplineBins": 0.001,
    # list of all elements
    "elements": ['C', 'O', 'H'],
  
    # Embeddings are specified for each individual elements,
    # all parameters could be distinct for different species
    # possible keywords: ALL, UNARY, elements: C, O, H
    "embeddings": {
      "ALL": {
        "npot": 'FinnisSinclairShiftedScaled',
        "fs_parameters": [ 1, 1], # linear embedding function: 1*rho_1^1
        "ndensity": 1,
        
        # core repulsion parameters
        "rho_core_cut": 3000,
        "drho_core_cut": 150
      },
    },
  
    # Bonds are specified for each possible pairs of elements
    # One could use keywords: ALL
    # possible keywords: ALL, UNARY, BINARY, elements pairs as CO, CC, HO, etc...
    "bonds": {
      "ALL": {
        "radbase": "ChebExpCos",
        "radparameters": [5.25],
  
        ## outer cutoff
        "rcut": 5,
        "dcut": 0.01,
  
        ## inner cutoff  [r_in - delta_in, r_in] - transition region from ACE to core-repulsion
        # at r < r_in-delta_in - no ACE interaction, only core-repulsion
        "r_in": 1.0,
        "delta_in": 0.5,
  
  
        ## core-repulsion parameters `prefactor` and `lambda` in
        ## prefactor*exp(-lambda*r^2)/r, > 0 only if r < r_in - delta_in
        "core-repulsion": [100.0, 5.0],
      },
      
      ## BINARY overwrites ALL settings when they are repeated
      "BINARY": {
        "radbase": "ChebPow",
        "radparameters": [6.25],
  
        ## cutoff may vary for different bonds
        "rcut": 5.5,
        "dcut": 0.01,
  
        ## inner cutoff, applied in a range [r_in - delta_in, r_in]
        "r_in": 1.0,
        "delta_in": 0.5,
  
        ## core-repulsion parameters `prefactor` and `lambda` in
        ## prefactor*exp(-lambda*r^2)/r,> 0 only if r < r_in - delta_in
        "core-repulsion": [10.0, 5.0],
      }
    },
  
    ## possible keywords: ALL, UNARY, BINARY, TERNARY, QUATERNARY, QUINARY,
    ##  element combinations as (Al,Al), (Al, Ni), (Al, Ni, Zn), etc...
    "functions": {
      # "number_of_functions_per_element": 1000,
      "ALL": {
          "nradmax_by_orders": [ 8, 8, 4, 3,2],
          "lmax_by_orders"   : [ 0, 4, 3, 2,1] 
      }
    }
  })

# Prepare linear ACE dataset

In [11]:
train_ds = LinearACEDataset(bconf, df_train)
test_ds = LinearACEDataset(bconf, df_test)

Construct design matrix for train and test, by default - in parallel with 4 workers/processes. 
Set verbose=False to suppress output

In [12]:
train_ds.construct_design_matrix(verbose=True, max_workers=8)
test_ds.construct_design_matrix(verbose=True, max_workers=8)

Create linear ACE fit class, provide train dataset

In [13]:
linear_fit = LinearACEFit(train_dataset=train_ds)

Call fit method, by default it uses Ridge from sklearn

In [14]:
linear_fit.fit()

Ridge(alpha=1e-05, copy_X=False, fit_intercept=False, random_state=42)

Compute error metrics on train and test (in eV/atom and eV/A)

In [15]:
linear_fit.compute_errors(train_ds)

{'epa_mae': 4.949501871574124,
 'epa_rmse': 4.949545207951954,
 'f_comp_mae': 0.8584654056412443,
 'f_comp_rmse': 1.1733577155363832}

In [16]:
linear_fit.compute_errors(test_ds)

{'epa_mae': 4.9470586543221575,
 'epa_rmse': 4.947103056023621,
 'f_comp_mae': 0.8874850802775792,
 'f_comp_rmse': 1.2105368917406725}

## Trained basis export and usage
Get fitted basis and use in PyACECalculator

In [20]:
linear_fit

In [23]:
basis = linear_fit.get_bbasis()
calc = PyACECalculator(basis)
e_pred, f_pred = linear_fit.predict(test_ds, reshape_forces=True)
e_pred

KeyboardInterrupt: 